# Experimento 10 : Módulo de inyección de datos.

En este experimento se pretende crear una red neuronal basada en los datos contextuales adheridos al conjunto de datos objeto de este proyecto.

El análisis de imágenes es una buena herramienta, pero trabajar con datos contextuales puede aumentar la precisión del diagnóstico.

En este caso es importante evaluar el rendimiento del modelo neuronal, esto es decisivo de cara a decidir si proseguir o no con el problema. Como ya se ha constatado en  experimentos anteriores uno de los factores determinantes para que un ensamblaje de modelos funcione es que ambos modelos tienen una precisión similar.

Basado en lo anterior en este estudio se plantea la siguiente hipótesis:
- Una red neuronal sencilla basada en información contextual puede conseguir un desempeño óptimo y posteriormente usarse en un ensamblaje de modelos.


## Librerías usadas.

In [ ]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math 
from glob import glob
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn.utils import shuffle

## Definición de rutas

In [ ]:
#Rutas de los datos.
 
data_dir = os.path.dirname(os.path.realpath("../TFG/Datos/HAM10000_metadata.csv"))



csv_path = os.path.realpath(data_dir + "/HAM10000_metadata.csv")

#Variables globales

clases = 7


print(data_dir)

print(csv_path)



## Creación del marco de datos.

In [ ]:
def combineData(data_dir):
    all_image_path = glob(os.path.join(data_dir, '*', '*'))
    imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
    return imageid_path_dict

#Inicializando el dataFrame
data_dir_mascara_binaria = os.path.dirname(os.path.realpath("../TFG/DatosMascaraBinaria/..."))
data_dict_mask = combineData(data_dir_mascara_binaria)

dataFrame=pd.read_csv(csv_path)

#Mezclando carpetas.

all_image_path = glob(os.path.join(data_dir, '*', '*'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}

# Inicializando diccionario de categorías

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

#Añadiendo columnas al dataFrame para que sea más legible.

dataFrame['path'] = dataFrame['image_id'].map(imageid_path_dict.get)
dataFrame['mask_path'] = dataFrame['image_id'].map(data_dict_mask.get)
dataFrame['cell_type'] = dataFrame['dx'].map(lesion_type_dict.get)

dataFrame['cell_type_idx'] = pd.Categorical(dataFrame['cell_type']).codes
dataFrame.head()
df = dataFrame

In [ ]:
df.head()

## Preparación de datos

In [ ]:
def plotMissingParams(data,color="blue",val=False):
    plt.figure(figsize = (13,5))
    if val:
        plt.bar(data.columns, data.isnull().sum(), color = color)
    else:
        plt.bar(data.columns, data.isna().sum(),color=color)
    plt.xlabel("Nombre de las columnas")
    plt.ylabel("Número celdas vacías")
    plt.show()

In [ ]:
plotMissingParams(dataFrame,color="green")

In [ ]:
def drop_not_used_elements(dataFrame):
    dataFrame.drop("lesion_id", axis = 1,inplace = True)
    dataFrame.drop("image_id", axis = 1,inplace = True)
    dataFrame.drop("dx", axis = 1,inplace = True)
    dataFrame.drop("path", axis = 1,inplace = True)
    dataFrame.drop("mask_path", axis = 1,inplace = True)
    dataFrame.drop("cell_type", axis = 1,inplace = True)

def map_values(dataFrame,element,value_map):
    dataFrame[element] = dataFrame[element].map(value_map)

def map_age(dataFrame):
    dataFrame.loc[ dataFrame['age'] <= 16, 'age'] = 0
    dataFrame.loc[(dataFrame['age'] > 16) & (dataFrame['age'] <= 32), 'age'] = 1
    dataFrame.loc[(dataFrame['age'] > 32) & (dataFrame['age'] <= 48), 'age'] = 2
    dataFrame.loc[(dataFrame['age'] > 48) & (dataFrame['age'] <= 64), 'age'] = 3
    dataFrame.loc[ dataFrame['age'] > 64, 'age'] = 4

def fill_all_empties_values(dataFrame):
    
    # Calculamos la media de las edades y sustituimos los valores vacios por el resultado.
    dataFrame["age"] = dataFrame["age"].fillna(dataFrame["age"].mean())
    
    #   Sustituimos los valores nulos "female". Esto es debido a que existen
    # menos mujeres que hombres y pretendemos balancear el conjunto de datos.
    dataFrame['sex'].fillna("female",inplace=True)
    
    return dataFrame
    

    
    
    
    
def map_all_values(dataFrame): 
    lesion_evaluation_type_map = {"histo": 1, "follow_up": 2, "consensus": 3, "confocal": 4}
    localization_map = {"acral": 0,"back": 1,"lower extremity" : 2,"trunk": 3,"upper extremity":4,"abdomen":5,"face":6,"chest":7,"foot":8,"unknown":9,"neck":10,"scalp":11,"hand":12,"ear":13,"genital":14}
    gender_mapping = {"male": 1, "female":0}    
    
    map_values(dataFrame,'localization',localization_map)
    map_values(dataFrame,'dx_type',lesion_evaluation_type_map)
    map_values(dataFrame,'sex',gender_mapping)
    map_age(dataFrame)

    
def data_inyection_module(dataFrame):
    dataFrame = fill_all_empties_values(dataFrame)
    map_all_values(dataFrame)
    drop_not_used_elements(dataFrame)
    print(dataFrame.head())
    data = shuffle(dataFrame, random_state=123)
    data.fillna(0,inplace=True)
    df_balanced = pd.DataFrame()
    for cat in df['cell_type_idx'].unique():
        temp = resample(df[df['cell_type_idx'] == cat],
                        replace=True,     # sample with replacement
                        n_samples=1000,   # to match majority class
                        random_state=123) # reproducible results
        # Combine majority class with upsampled minority class
        df_balanced = pd.concat([df_balanced, temp])
    Y_data = df_balanced['cell_type_idx']
    Y_data.fillna(0,inplace=True)
    X_data = df_balanced.drop("cell_type_idx", axis = 1)
    X_data.fillna(0,inplace=True)
    return df_balanced,X_data,Y_data




In [ ]:
dataFrame,X_data,Y_data = data_inyection_module(dataFrame)

In [ ]:
df = resample(dataFrame[df['cell_type_idx']==0],
                      replace=True,     
                      n_samples=1000,  
                      random_state=123)

In [ ]:
df = dataFrame

In [ ]:
dataFrame['cell_type_idx'].value_counts()

In [ ]:
print(len(X_data))
print(len(Y_data))

## Estudio de la propuesta de red y ejecución

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from numpy import set_printoptions
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier



#Clasificador basado en K-Best
test = SelectKBest(score_func=f_classif, k='all')
fit = test.fit(X_data, Y_data)
print(["dx_type","age","sex","location"])
print(fit.scores_)

#Clasificador basado en arboles
model = ExtraTreesClassifier(n_estimators=100)
model.fit(X_data, Y_data)
print(model.feature_importances_)

#Clasificador basado en regresión logística
model = LogisticRegression(solver='liblinear')
rfe = RFE(model, 1)
fit = rfe.fit(X_data, Y_data)
print("Num Features: %d" % fit.n_features_)
print("Feature Ranking: %s" % fit.ranking_)

In [ ]:
X_data=X_data.drop("sex", axis = 1)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_data,Y_data,test_size=0.40,random_state=123)
x_test,x_validation,y_test,y_validation = train_test_split(x_test,y_test,test_size=0.50,random_state=123)

In [ ]:
def decodeBlock(n_neurons,bachnorm = True,dropout = 0.2):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(n_neurons))

    if bachnorm:
        model.add(tf.keras.layers.BatchNormalization())

    if dropout > 0:
        model.add(tf.keras.layers.Dropout(dropout))
  
    model.add(tf.keras.layers.PReLU())

    return model
def codeBlock(n_neurons,dropout = 0.2):

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(n_neurons))

    if dropout > 0:
        model.add(tf.keras.layers.Dropout(dropout))
  
    model.add(tf.keras.layers.PReLU())

    return model


In [ ]:
inputs = tf.keras.layers.Input(shape=[3])

x = inputs

x = codeBlock(16)(x)
x = codeBlock(64)(x)
x = decodeBlock(16,bachnorm=False,dropout = 0.15)(x)

last = tf.keras.layers.Dense(units = 7, activation = 'softmax')(x)

model = tf.keras.Model(inputs=inputs,outputs=last)
model.summary()

In [ ]:


callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                           patience = 12,
                                           mode = 'min')

ams_grad = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
    name='Adam'
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=0.000001)

In [ ]:
#Se ha decidido realizar un test rápido con un modelo de Keras para asegurar que
#el comportamiento de nuestra red no es fallo de la arquitectura de esta

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

print(knn.score(x_test,y_test))


In [ ]:
for i in range(0,10):
    model.compile(optimizer = ams_grad, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])
    history = model.fit(x_train, y_train ,validation_data=(x_validation, y_validation),epochs = 200,batch_size = 16,verbose = 0,callbacks = [callback,reduce_lr])
    model.evaluate(x_test,y_test)
print("finish")